In [1]:
# !pip install tensorflow==2.0.0-alpha0

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn import model_selection, metrics
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
import seaborn as sns
import math
import keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import Sequential, Input, Model, losses, optimizers
import glob


import os
print(os.listdir("."))

['tf2_model2-Boosted.ipynb', '.ipynb_checkpoints', 'test.csv', 'train.csv', 'Keras_-_PUBG.ipynb', 'sample_submission.csv', 'tf2_model1.ipynb', 'estimator_submission.csv', 'estimator_api.ipynb', 'PUBG_submit.ipynb', 'Esti_works.ipynb', 'tf2_model3-Keras.ipynb', 'EDA.ipynb', 'Untitled.ipynb']


Using TensorFlow backend.


In [3]:
def load_data(train_path='train.csv', test_path='test.csv'):
    train = pd.DataFrame.from_csv(train_path, index_col="Id")
    test = pd.DataFrame.from_csv(test_path, index_col="Id")
    
    return train, test

In [4]:
train, test = load_data()

/home/lazukav/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  
/home/lazukav/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
def preprocess(data):
    data.dropna(axis=0, inplace=True)
    if 'winPlacePerc' in data:
        data.winPlacePerc = np.where(data.winPlacePerc >= 0.5 , 1, 0)
        return data[['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals', 'winPlacePerc']]
    return data[['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals']]

In [6]:
train = preprocess(train)
test = preprocess(test)

In [7]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(train.drop('winPlacePerc', axis=1), train.winPlacePerc, test_size=0.2, shuffle=None)

In [8]:
print ("Training Shape:")
print ("\tFeature: ", X_train.shape)
print ("\tTarget: ", y_train.shape)
print ("Validation Shape:")
print ("\tFeature: ", X_test.shape)
print ("\tTarget: ", y_test.shape)

Training Shape:
	Feature:  (3557572, 6)
	Target:  (3557572,)
Validation Shape:
	Feature:  (889393, 6)
	Target:  (889393,)


In [9]:
model = Sequential([
    Dense(128, activation=tf.nn.relu,input_shape=(6, )),
    Dense(64, activation=tf.nn.sigmoid),
    Dropout(0.5),
    Dense(128, activation=tf.nn.relu),
    Dense(64, activation=tf.nn.sigmoid),
    Dense(1, activation=tf.nn.sigmoid)
])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               896       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 25,793
Trainable params: 25,793
Non-trainable params: 0
____________________________________________________

In [11]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer = optimizers.Adam(
        lr = 0.001,
        decay = 0.0,
    ),
    metrics = ['accuracy']
)

In [12]:
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = 500,
    epochs = 5,
    verbose = 1,
    validation_data = (X_test, y_test),
    callbacks = [tf.keras.callbacks.TensorBoard(log_dir="/tmp/model3/")]
)

Train on 3557572 samples, validate on 889393 samples
Epoch 1/5
3557572/3557572 [==============================] - 41s 11us/sample - loss: 0.4655 - accuracy: 0.7895 - val_loss: 0.4562 - val_accuracy: 0.7955
Epoch 2/5
3557572/3557572 [==============================] - 39s 11us/sample - loss: 0.4568 - accuracy: 0.7938 - val_loss: 0.4531 - val_accuracy: 0.7963
Epoch 3/5
3557572/3557572 [==============================] - 39s 11us/sample - loss: 0.4552 - accuracy: 0.7943 - val_loss: 0.4530 - val_accuracy: 0.7951
Epoch 4/5
3557572/3557572 [==============================] - 41s 12us/sample - loss: 0.4545 - accuracy: 0.7947 - val_loss: 0.4529 - val_accuracy: 0.7954
Epoch 5/5
3557572/3557572 [==============================] - 39s 11us/sample - loss: 0.4542 - accuracy: 0.7947 - val_loss: 0.4533 - val_accuracy: 0.7945


In [13]:
prediction = model.predict(test)

In [14]:
output = pd.DataFrame(
    [(Id, 1) if pred >= 0.5 else (Id, 0)  for Id, pred in zip(test.index, prediction)],
    columns = ['Id', 'winPlacePerc']
)

In [15]:
output.to_csv("submission3.csv", index = False)